## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-24-14-51-29 +0000,nyt,A Paris Art Event So Exclusive That Breaking I...,https://www.nytimes.com/2025/10/24/arts/design...
1,2025-10-24-14-50-43 +0000,bbc,'It became one of the children' – Kenyan famil...,https://www.bbc.com/news/articles/c4gj4x4939qo...
2,2025-10-24-14-48-19 +0000,nyt,Trump Administration Live Updates: President A...,https://www.nytimes.com/live/2025/10/24/us/tru...
3,2025-10-24-14-47-22 +0000,nyt,Trump Says He’s Cutting Off Trade Negotiations...,https://www.nytimes.com/2025/10/23/us/politics...
4,2025-10-24-14-46-42 +0000,nypost,White House trolls Dems over ballroom drama wi...,https://nypost.com/2025/10/24/us-news/white-ho...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2379/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
18,trump,63
27,off,17
133,new,16
56,will,16
24,canada,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
190,2025-10-24-03-21-49 +0000,cbc,Trump claims Canada 'CHEATED AND GOT CAUGHT' w...,https://www.cbc.ca/news/world/trump-trade-nego...,153
2,2025-10-24-14-48-19 +0000,nyt,Trump Administration Live Updates: President A...,https://www.nytimes.com/live/2025/10/24/us/tru...,152
191,2025-10-24-03-16-11 +0000,nypost,Trump terminates trade talks with Canada over ...,https://nypost.com/2025/10/23/us-news/trump-te...,133
70,2025-10-24-12-13-55 +0000,cbc,LIVE UPDATES | Carney says Canada prepared to ...,https://www.cbc.ca/news/canada/livestory/live-...,132
235,2025-10-23-23-29-06 +0000,nypost,Ford warns devasting plant fire will hurt prof...,https://nypost.com/2025/10/23/business/ford-wa...,131


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
190,153,2025-10-24-03-21-49 +0000,cbc,Trump claims Canada 'CHEATED AND GOT CAUGHT' w...,https://www.cbc.ca/news/world/trump-trade-nego...
235,68,2025-10-23-23-29-06 +0000,nypost,Ford warns devasting plant fire will hurt prof...,https://nypost.com/2025/10/23/business/ford-wa...
269,58,2025-10-23-21-24-00 +0000,wsj,Lawmakers failed to provide a last-minute repr...,https://www.wsj.com/politics/policy/air-traffi...
54,50,2025-10-24-13-06-00 +0000,wsj,Russia’s Central Bank Cuts Key Rate as New San...,https://www.wsj.com/articles/russias-central-b...
4,47,2025-10-24-14-46-42 +0000,nypost,White House trolls Dems over ballroom drama wi...,https://nypost.com/2025/10/24/us-news/white-ho...
67,45,2025-10-24-12-24-07 +0000,nypost,Wild video shows security shoot at U-Haul truc...,https://nypost.com/2025/10/24/us-news/video-sh...
41,45,2025-10-24-13-31-00 +0000,nyt,Gasoline prices drove up inflation in Septembe...,https://www.nytimes.com/live/2025/10/24/busine...
33,43,2025-10-24-13-44-56 +0000,nypost,Six dead after US strike in Caribbean targets ...,https://nypost.com/2025/10/24/us-news/6-dead-a...
30,41,2025-10-24-13-54-28 +0000,nypost,"‘Sex warfare’: China, Russia send hot women to...",https://nypost.com/2025/10/24/business/china-r...
366,36,2025-10-23-15-32-37 +0000,nypost,Zohran Mamdani’s dad allegedly tried to silenc...,https://nypost.com/2025/10/23/world-news/zohra...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
